### Import Libraries

In [ ]:
import tda
import pandas as pd
import json
import datetime
import httpx
import math
import time
import random

from tqdm.notebook import tqdm, trange

### Get Date

In [ ]:
now = datetime.datetime.now()
yesterday = (now - datetime.timedelta(days = 1)).strftime('%Y%m%d')
today = (now).strftime('%Y%m%d')

current_month = now.month
current_year = now.year

print('Today:\t\t', today)
print('Yesterday:\t', yesterday)
print('Month:\t\t', current_month)
print('Year:\t\t', current_year)

### Create Client

In [ ]:
#read file with username, pass, server
with open('tda.json') as f:
    data = json.load(f)

API_KEY = data['api_key']
REDIRECT_URI = data['redirect_uri']
TOKEN_PATH = data['token_path']

def make_webdriver():
    # Import selenium here because it's slow to import
    from selenium import webdriver
    import atexit

    driver = webdriver.Chrome()
    atexit.register(lambda: driver.quit())
    return driver


# Create a new client
client = tda.auth.easy_client(
    API_KEY,
    REDIRECT_URI,
    TOKEN_PATH,
    make_webdriver)

In [ ]:
# Load S&P 500 composition from documentation
SP500_URL = "https://tda-api.readthedocs.io/en/latest/_static/sp500.txt"
sp500 = httpx.get(
    SP500_URL, headers={
        "User-Agent": "Mozilla/5.0"}).read().decode().split()

#symbols no longer in SP500
sp500.remove('AGN')
sp500.append('FAS')
sp500.append('GDRX')

In [ ]:
#create empty df
d = {'Symbol':[],
     'Name':[],
     'divDate':[],
     'divAmount':[],
     'divYield':[],
     'closePrice':[],
     'lastPrice':[],
     'expDate1':[],
     'daysUnitlExp1':[],
     'strikePrice1':[],
     'callPremium':[]}
df = pd.DataFrame(data = d)

df

In [ ]:
def get_standard_quote(symb):
    symb = symb.upper()
    #get quote
    resp = client.get_quote(symb)
    assert resp.status_code == 200, resp.raise_for_status()
    
    return resp.json()

def get_call_chain(symb):
    resp = client.get_option_chain(symb,
                                   contract_type = tda.client.Client.Options.ContractType.CALL,
                                   strike_range = tda.client.Client.Options.StrikeRange.OUT_OF_THE_MONEY,
                                   strike_count = 5,
                                   from_date = div_date,
                                   to_date = div_date + datetime.timedelta(days = 30)
                                    )
    assert resp.status_code == 200, resp.raise_for_status()
    
    return resp.json()

def myround5(x, base=5):
    return base * math.ceil(x/base)

def myround10(x, base=10):
    return base * math.ceil(x/base)

In [ ]:
p_bar = tqdm(sp500)

for x in p_bar:
    p_bar.set_description(f'Working on "{x}"')
    
    std_resp_json = get_standard_quote(x)
    exp_date1 = None
    strike_price1 = None
    call_premium = None
    days_until_exp1 = None
    
    if len(std_resp_json) > 0:
        div_amount = std_resp_json[x]['divAmount']/4
        div_yield = std_resp_json[x]['divYield']
        div_date = std_resp_json[x]['divDate']
        close_price = std_resp_json[x]['closePrice']
        last_price = std_resp_json[x]['lastPrice']
        name = std_resp_json[x]['description']
        
        #get option chain if company pays dividend
        if div_amount > 0:
            #convert div_date to datetime
            div_date = datetime.datetime.fromisoformat(std_resp_json[x]['divDate'])
            time.sleep(random.randrange(3))
            
            #only look for option chains with div date in the future
            if div_date > now:
                call_resp_json = get_call_chain(x)
                
                if call_resp_json['status'] == 'SUCCESS':

                    #list of expirations
                    exp_date_list = [a for a in call_resp_json['callExpDateMap'].keys()]

                    exp_date1a = exp_date_list[0]
                    exp_date1b = exp_date_list[0].split(':')[0]
                    days_until_exp1 = exp_date_list[0].split(':')[1]

                    str_price_list = [float(str_pr) for str_pr in call_resp_json['callExpDateMap'][exp_date1a].keys()]
                    for i, j in enumerate(str_price_list):
                        if j > last_price:
                            strike_price1 = j
                            break

                    try:
                        call_premium = call_resp_json['callExpDateMap'][exp_date1a][str(strike_price1)][0]['bid']
                    except KeyError:
                        pass
        
    symb_div = {'Symbol':x,
                'Name':name,
                'divDate':div_date,
                'divAmount':div_amount,
                'divYield':div_yield,
                'closePrice':close_price,
                'lastPrice':last_price,
                'expDate1':exp_date1b,
                'daysUnitlExp1':days_until_exp1,
                'strikePrice1':strike_price1,
                'callPremium':call_premium
               }

    symb_df = pd.DataFrame.from_dict(symb_div, orient = 'index').T
        
    df = pd.concat([df,symb_df])

In [ ]:
df.sample(5)

In [ ]:
#convert data types
df['divDate'] = pd.to_datetime(df['divDate'])
df['expDate1'] = pd.to_datetime(df['expDate1'])
df['divAmount'] = df['divAmount'].astype(float)
df['lastPrice'] = df['lastPrice'].astype(float)
df['daysUnitlExp1'] = df['daysUnitlExp1'].astype(float)
df['strikePrice1'] = df['strikePrice1'].astype(float)
df['callPremium'] = df['callPremium'].astype(float)

#calculate cols
df['potentialGainCallExecuted'] = (df['divAmount'] + df['callPremium'] + (df['strikePrice1'] - df['lastPrice'])) * 100
df['cost'] = (df['lastPrice'] - df['callPremium']) * 100

#return call executed
df['returnCallExecuted'] = df['potentialGainCallExecuted'] / df['cost']
df['annualizedReturnCallExecuted'] = (365/df['daysUnitlExp1']) * df['returnCallExecuted']

#return call not executed and stock does not change
df['returnCallNotExecuted'] = (df['divAmount'] + df['callPremium']) * 100 / df['cost']
df['annualizedReturnCallNotExecuted'] = (365/df['daysUnitlExp1']) * df['returnCallNotExecuted']

#break even
df['breakEven'] = (df['cost'] - (df['divAmount']*100))/100

df.reset_index(inplace = True, drop = True)
df.info()

In [ ]:
#df_div = df[~df['expDate1'].isna()]
df_div = df[df['divAmount'] > 0]
df_covered_call = df[df['strikePrice1'] > 0]

#export to excel
df_div.to_excel('Output/DividendStocks_' + today + '.xlsx', index = False)
df_covered_call.to_excel('Output/CoveredCalls_' + today + '.xlsx', index = False)
df_div.head()